Start by importing all the modules to be used

In [ ]:
import tango
import time
import json
import os

Set up device proxies for our tango devices and put them into admin mode (allowing them to be used)

In [ ]:
# create device proxies and put devices in Online, not simulated mode
sr=tango.DeviceProxy('low-mccs/subrack/01')
sr.adminMode=0
sr.simulationMode=1

tpm1=tango.DeviceProxy("low-mccs/tile/0001")
tpm1.adminMode = 0
tpm1.simulationMode = 0

daq = tango.DeviceProxy("low-mccs-daq/daqreceiver/001")
daq.adminMode = 0
daq.simulationMode = 1

# turn on subrack and wait for ON status
sr.On()
while sr.state() != tango._tango.DevState.ON:
    time.sleep(1)

# turn on DAQ and wait for ON status
daq.On()
while daq.state() != tango._tango.DevState.ON:
    time.sleep(1)

# turn on TPM and wait for initialization. 
tpm1.On()
while tpm1.tileprogrammingstate != "Initialised" and \
	tpm1.tileprogrammingstate != "Synchronised":
	time.sleep(1)

Set up the data stream on the tile and daq

In [ ]:
# Start TPM acquisition and wait for ADC stream to start
tpm1.startacquisition('{}')
while tpm1.tileprogrammingstate != "Synchronised":
	time.sleep(1)

#configure networking between TPM and DAQ.
dst_tpm_port = 4660
dst_tpm_ip = "10.0.10.2"
lmc_download_dict = {"Mode": "10g", "DspIP": dst_tpm_ip, "DstPort": dst_tpm_port}
tpm1.SetLmcDownload(json.dumps(lmc_download_dict))
daq.Configure('{"receiver_ports": [dst_tpm_port], "receiver_interface": "eth0", "receiver_ip": dst_tpm_ip,}')
daq.SetConsumers("RawData")

Start the daq

In [ ]:
#bug currently in the Start command
daq.Start()

data_to_send = {"data_type": "raw"}
tpm1.SendDataSamples(json.dumps(data_to_send))

Prove that the daq has started writing to files 

In [ ]:
received_data_mode, received_data_result = daq.dataReceivedResult
data = json.loads(received_data_result)
file_name = data['filename']

if (os.path.exists(file_name)):
    print("Data is being streamed to file")
else:
    print("Data ack has failed")

Show that we can monitor health attributes from jupyter notebook

In [ ]:
tgm -w 1200 low-mccs-spshw/tile/0001/voltage